In [10]:
import os
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup
import pandas as pd

##############################################################  ############
##################### variable related selenium ##########################
##########################################################################
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')

driver = webdriver.Chrome('./chromedriver')
# chromedriver_path = "chromedriver"
# driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기

rating_df = pd.DataFrame()
restaurant_df = pd.DataFrame()

def main():
    global driver, load_wb, review_num

    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/')  # 주소 가져오기

    # 검색할 목록
    lists = ['종로구 맛집', '중구 맛집', '용산구 맛집', '성동구 맛집', '광진구 맛집', '동대문구 맛집', '중랑구 맛집', '성북구 맛집', '강북구 맛집']
    for list in lists:
        search(list)

    driver.quit()
    print("finish")


def search(place):
    global driver

    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER) # 더보기
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, place_lists)
    search_area.clear()

    # 전체 페이지
    while True:
        try:
#             driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
#             sleep(1)
            # 2~ 5페이지 읽기
            for i in range(2, 6):
                # 페이지 넘기기
                xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
                driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
                sleep(1)

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list
                
                crawling(place, place_lists)
                
                # 다음 페이지 넘기기
                if i==5:
                    driver.find_element_by_xpath('//*[@id="info.search.page.next"]').send_keys(Keys.ENTER)

        except ElementNotInteractableException:
            print('end page')
            break
#         finally:
#             search_area.clear()


def crawling(place, place_lists):
    """
    페이지 목록을 받아서 크롤링 하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    """
    
    global restaurant_df

    while_flag = False
    for i, place in enumerate(place_lists):
        # 광고에 따라서 index 조정해야함
        #if i >= 3:
         #   i += 1

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address
        place_local = place.select('.info_item > .addr > .lot_number')[0].text
        place_category = place.select('.head_item > .subcategory')[0].text
        place_detail = place.select('.info_item > .contact> .moreview')[0].get('href') # place detail
        
        row = {"ItemID":place_name, "address": place_address, "local" : place_local, "category": place_category}
        row = pd.DataFrame(row, index=[1])
        restaurant_df = restaurant_df.append(row, ignore_index=True)
        
        driver.execute_script('window.open("about:blank", "_blank");')
        driver.switch_to.window(driver.window_handles[-1])
        driver.get(place_detail) # 상세정보 탭으로 변환
        sleep(1)

#         detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
#         driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
#         driver.switch_to.window(driver.window_handles[-1])  
        
        print('####', place_name)

        # 첫 페이지
        extract_review(place_name) # 리뷰 추출

        # 2-5 페이지
        idx = 3
        try:
            page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
            
            for i in range(page_num-1):
                # css selector를 이용해 페이지 버튼 누르기
                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                sleep(1)
                extract_review(place_name)
                idx += 1
            driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 5페이지가 넘는 경우 다음 버튼 누르기
            sleep(1)
            extract_review(place_name) # 리뷰 추출
        except (NoSuchElementException, ElementNotInteractableException):
            print("no review in crawling")

        # 그 이후 페이지
        while True:
            idx = 4
            try:
                page_num = len(driver.find_elements_by_class_name('link_page')) #페이지 수 찾기
                
                for i in range(page_num-1):
                    driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                    sleep(1)
                    extract_review(place_name)
                    idx += 1
                driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 10페이지 이상으로 넘어가기 위한 다음 버튼 클릭
                sleep(1)
                extract_review(place_name) # 리뷰 추출
                
            except (NoSuchElementException, ElementNotInteractableException):
                print("no review in crawling")
                break

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환


def extract_review(place_name):
    global driver, rating_df

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점
            user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
            timestamp = review.select(' div > span.time_write') #시간정보
            
            val = ''
            if len(comment) != 0:
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                else:
                    val = comment[0].text + '/0'
#                 print(val)
                
                
                try:
                    if(user_id == NULL and rating ==NULL):
                        row = {"ItemID":place_name, "UserID": None, "review" : None,
                               "Rating":None, "Timestamp":timestamp[0].text}
                        row = pd.DataFrame(row, index=[i])
                        rating_df = rating_df.append(row, ignore_index=True)
                        
                    elif(user_id == NULL):
                        row = {"ItemID":place_name, "UserID": None, "review" : None,
                               "Rating":None, "Timestamp":timestamp[0].text}
                        row = pd.DataFrame(row, index=[i])
                        rating_df = rating_df.append(row, ignore_index=True)
                        
                    elif(rating ==NULL):
                        row = {"ItemID":place_name, "UserID": None, "review" : None,
                               "Rating":None, "Timestamp":timestamp[0].text}
                        row = pd.DataFrame(row, index=[i])
                        rating_df = rating_df.append(row, ignore_index=True)
                        
                    else:
                        row = {"ItemID":place_name, "UserID":user_id[0].get('data-userid'), "review" : comment[0].text,
                               "Rating":rating[0].text.replace('점', ''), "Timestamp":timestamp[0].text}
                        row = pd.DataFrame(row, index=[i])
                        rating_df = rating_df.append(row, ignore_index=True)    
            
                except:
                    row = {"ItemID":place_name, "UserID":None, "review" : None,
                           "Rating": None, "Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)                
                
                
#                 try:
#                     row = {"ItemID":place_name, "UserID":user_id[0].get('data-userid'), "review" : comment[0].text,
#                            "Rating":rating[0].text.replace('점', ''), "Timestamp":timestamp[0].text}
#                     row = pd.DataFrame(row, index=[i])
#                     rating_df = rating_df.append(row, ignore_index=True)
            
#                 except:
#                     row = {"ItemID":place_name, "UserID":None, "review" : None,
#                            "Rating": None, "Timestamp":timestamp[0].text}
#                     row = pd.DataFrame(row, index=[i])
#                     rating_df = rating_df.append(row,ignore_index=True)
                
                
    else:
        print('no review in extract')
        ret = False

    return ret


if __name__ == "__main__":
    main()

#### 어니언 안국점
no review in crawling
#### 황생가칼국수


KeyboardInterrupt: 

In [44]:
rating_df.head()

,ItemID,UserID,review,Rating,Timestamp
0,조연탄,k9hsnf,헉 가게가 좀 부산스럽고 종업원분들이 극 인싸이싱거같아요 그거랑 별개로 맛은 정말 ...,5,2021.08.03.
1,조연탄,o9qv2m,어엄청 친절하셔유! 맛있어여 또 갈거예요,5,2021.06.25.
2,조연탄,1cbc382,,5,2021.06.23.
3,조연탄,13n8k41,찐맛집,5,2021.06.19.
4,조연탄,1fio525,내 최애 고기집,5,2021.06.15.


In [43]:
restaurant_df.head(20)

,ItemID,address,local,category
0,조연탄,서울 강서구 곰달래로60길 29,(지번) 화곡동 782-12,"육류,고기"
1,먹자대게,서울 양천구 목동서로 213 세신비젼프라자 1층 108호,(지번) 목동 923,"게,대게"
2,착한낙지 목동점,서울 양천구 국회대로 289,(지번) 목동 802-9,"해물,생선"
3,일미락 목동본점,서울 양천구 목동동로 226-16 1층,(지번) 목동 406-126,"육류,고기"
4,원조소금구이,서울 강서구 곰달래로60길 5 1층,(지번) 화곡동 781-12,"육류,고기"
5,엉털네숯불꼼장어,서울 양천구 등촌로 36,(지번) 목동 792-2,장어
6,더아리엘 목동점,서울 양천구 오목로 300 현대하이페리온2차 206동 지하1층 109~115호,(지번) 목동 961-1,뷔페
7,양천옥설렁탕,서울 양천구 등촌로 86,(지번) 목동 730-2,설렁탕
8,오목집 목동본점,서울 양천구 목동서로 155 목동파라곤 지하상가 45호,(지번) 목동 917,"족발,보쌈"
9,스타벅스 화곡DT점,서울 강서구 등촌로 57,(지번) 화곡동 772-67,커피전문점


In [40]:
rating_df.to_csv('rating_df.csv')

In [51]:
rating_df.to_csv('rating_df_test.csv', sep=',', na_rep='NaN', encoding='utf-8-sig')

In [48]:
rating_df.to_csv('rating_df_ko.csv', encoding='utf-8-sig')

In [39]:
restaurant_df.to_csv('restaurant_df.csv')

In [53]:
restaurant_df.to_csv('restaurant_df_test.csv', sep=',', na_rep='NaN', encoding='utf-8-sig')

In [49]:
restaurant_df.to_csv('restaurant_df_ko.csv' , encoding='utf-8-sig')

In [9]:
import os
import pandas as pd

from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

##############################################################  ############
##################### variable related selenium ##########################
##########################################################################
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')

driver = webdriver.Chrome('./chromedriver')
# chromedriver_path = "chromedriver"
# driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기

rating_df = pd.DataFrame() # 리뷰 저장 데이터 프레임
restaurant_df = pd.DataFrame() #음식점 정보 저장

def main():
    global driver, load_wb, review_num

    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/')  # 주소 가져오기

    # 검색할 목록
    lists = ['종로구 맛집', '중구 맛집', '용산구 맛집', '성동구 맛집', '광진구 맛집', '동대문구 맛집', '중랑구 맛집', '성북구 맛집', '강북구 맛집']
    for list in lists:
        search(list)
        
    driver.quit()
    print("finish")


def search(place):
    global driver

    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER) # 더보기
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, place_lists)
    search_area.clear()

    # 우선 더보기 클릭해서 2페이지
    try:
        driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
        sleep(1)

        # 2~ 5페이지 읽기
        for i in range(2, 6):
            # 페이지 넘기기
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
            sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list

            crawling(place, place_lists)

    except ElementNotInteractableException:
        print('not found')
    finally:
        search_area.clear()


def crawling(place, place_lists):
    """
    페이지 목록을 받아서 크롤링 하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    """

    while_flag = False
    for i, place in enumerate(place_lists):
        # 광고에 따라서 index 조정해야함
        #if i >= 3:
         #   i += 1

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address
        place_detail = place.select('.info_item > .contact> .moreview')[0].get('href') # place detail

        driver.execute_script('window.open("about:blank", "_blank");')
        driver.switch_to.window(driver.window_handles[-1])
        driver.get(place_detail) # 상세정보 탭으로 변환
        sleep(1)

#         detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
#         driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
#         driver.switch_to.window(driver.window_handles[-1])  
        
        print('####', place_name)

        # 첫 페이지
        extract_review(place_name) # 리뷰 추출

        # 2-5 페이지
        idx = 3
        try:
            page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
            
            for i in range(page_num-1):
                # css selector를 이용해 페이지 버튼 누르기
                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                sleep(1)
                extract_review(place_name)
                idx += 1
            driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 5페이지가 넘는 경우 다음 버튼 누르기
            sleep(1)
            extract_review(place_name) # 리뷰 추출
        except (NoSuchElementException, ElementNotInteractableException):
            print("no review in crawling")

        # 그 이후 페이지
        while True:
            idx = 4
            try:
                page_num = len(driver.find_elements_by_class_name('link_page')) #페이지 수 찾기
                
                for i in range(page_num-1):
                    driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                    sleep(1)
                    extract_review(place_name)
                    idx += 1
                driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 10페이지 이상으로 넘어가기 위한 다음 버튼 클릭
                sleep(1)
                extract_review(place_name) # 리뷰 추출
            except (NoSuchElementException, ElementNotInteractableException):
                print("no review in crawling")
                break

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환


def extract_review(place_name):
    global driver

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점

            
            val = ''
            if len(comment) != 0:
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                else:
                    val = comment[0].text + '/0'
                print(val)

    else:
        print('no review in extract')
        ret = False

    return ret


if __name__ == "__main__":
    main()

#### 어니언 안국점
아이랑 왔는데 친절하시고 서비스 정말 좋으시네요. 아이가 마실 딸기 요거트 시키려는데 직원분이 먼저 아이용 맛보기 샘플 무료로 제공중인데, 아이가 많이 마시는 편이 아니라면 그걸로도 양이 충분할 수 있을지 물어보셔서, 그렇다고 말씀드리며 감사히 무료 샘플 받았어요. 샘플 사이즈가 저희 아이에게 딱이었네요 ㅎㅎ 올리브 베이컨 빵이랑 앙셀 슈슈도 시켰는데 둘 .../5
/3
아침 일찍가면 사람 없어서 좋아요 카페메뉴 가격대가 높은편./4
멋진 장소 친절한 직원 그러나 벌레가 많음/4
공주밤빵 검정크런치 맛이 좋지는 않아요/1
장소가 넓고 분위기가 좋아요. 대체로 메뉴가 맛있어요/4
시오빵 너무맛있음/5
커피와 빵의 맛으로 볼 때 요즘 이 만한 집이 어디 한 둘이겠는가 그러나 대체 불가한 이 곳에 꼭 가야할 이유는 이 마루에 있지않을까 종로 한복판에 왜색짙은 까페와 식당이 정체성없이 난립될 때 정통 한옥의 재해석을 그 어느곳보다 완벽히 해내 서울의 아름다움을 한층 부각시킨다. 사브작되는 앞마당과 한낮 바람도 식어 흐르는 이 마루에서 읽히는 휴식은 최고의 힐.../5
사람 넘많아... 순환 안 돼고 정신ㄹ없음 /3
어니언은 어느 지점이나 꽤 친절한 것 같아 좋아요 빵도 맛있고 음료도 맛있고 굳  근데 사람 너무 많음…. /4
앙버터 맛집, 멀리까지 빵 사러 가기 힘들었는데 여기서 살 수 있어서 좋아요!! /5
맛있어요^^ 산미 부분은 조금 개선 해야 할듯해요ㅎ/5
빵도 넘 맛있고 직원분들이 친절해서 너무 좋음!아인슈페너 맛있다던데 담에 먹어봐야게씀 ㅎㅎ/5
/5
주말이라 조금 대기했지만 안내해주시는 직원분들이 친절하시고 자리나면 바로바로 배치해주셔서 좋았어요~ 빵도 맛있었구요!! 커피맛은 약간 아쉽네요^^;;/4
커피가 너무 맛이 없음. 산미있대서 기대했는데 산미도 없고 그렇다고 고소한 맛도 없이 텁텁 + 밍밍. 빵은 그럭저럭인데 전체적으로 돈이 아깝다 싶을 정도로 맛이 없음. 분위기는 좋음. 그냥 참새 구경하면서 한옥 배경으로 사진 찍기 좋은

KeyboardInterrupt: 